In [ ]:
import os
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy import ndimage

import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Conv2DTranspose

In [ ]:
from numpy.random import seed

In [ ]:
import tensorflow

In [ ]:
# zawsze pamietać o seed
tensorflow.random.set_seed(42)
seed(42)

In [ ]:
# shape
width = 968
height = 1292

# reaad only glaukolit
skalka = 'glaukolit'
all_glau = [i for i in glob.glob(os.getcwd() + '/maski/'+ '*{}*'.format(skalka))]

# reaad only mika
skalka = 'mika'
all_mika = [i for i in glob.glob(os.getcwd() + '/maski/'+ '*{}*'.format(skalka))]

# reaad only mika
skalka = 'kwarc'
all_kwarc = [i for i in glob.glob(os.getcwd() + '/maski/'+ '*{}*'.format(skalka))]

In [ ]:
# nums files
dif = []
for filename in all_glau:
    if filename[43] not in dif:
        dif.append(filename[43])

In [ ]:
def writer(nums):
    for num in nums:
        mika = np.zeros((width, height), np.uint8)
        glaukolit = np.zeros((width, height), np.uint8)
        kwarc = np.zeros((width, height), np.uint8)
        for filename in all_glau:
            c = filename[43] 
            if c == num:
                img = cv2.imread(filename, 0)
                glaukolit = cv2.bitwise_or(glaukolit, img)
        for filename in all_mika:
            c = filename[43] 
            if c == num:
                img = cv2.imread(filename, 0)
                mika = cv2.bitwise_or(mika, img)
        for filename in all_kwarc:
            c = filename[43] 
            if c == num:
                img = cv2.imread(filename, 0)
                kwarc = cv2.bitwise_or(kwarc, img)
        cv2.imwrite('masks/glaukolit_{}.tif'.format(num), glaukolit)
        cv2.imwrite('masks/mika_{}.tif'.format(num), mika)
        cv2.imwrite('masks/kwarc_{}.tif'.format(num), kwarc)

In [ ]:
writer(dif)

In [ ]:
train_masks  = []
skalka = 'glaukolit'
mi = [i for i in glob.glob(os.getcwd() + '/masks/'+ '*{}*'.format(skalka))]
for mask_path in mi:
    mask = cv2.imread(mask_path, 0)
    mask = cv2.resize(mask, (height, width))
    thresh = 1
    mask = cv2.threshold(mask, thresh, 255, cv2.THRESH_BINARY)[1]
    train_masks.append(mask)

In [ ]:
train_masks = np.array(train_masks)

In [ ]:
train_images  = []
im = [i for i in glob.glob(os.getcwd() + '/images/'+ '*.jpg')]
del im[0]
for img_path in im:
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
    #img = cv2.resize(img, (SIZE, SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    train_images.append(img)

In [ ]:
len(train_images)

In [ ]:
len(train_masks)

In [ ]:
train_images = np.array(train_images)

In [ ]:
# model
activation = 'sigmoid'
feature_extractor = Sequential()

feature_extractor.add(Conv2D(32, 3, activation = activation, padding = 'same', input_shape = (width, height, 3)))
feature_extractor.add(Conv2D(32, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))

In [ ]:
X_train = train_images
y_train = train_masks
y_train = np.expand_dims(y_train, axis=3)

In [ ]:
X = feature_extractor.predict(X_train)

In [ ]:
feature_extractor.compile()

In [ ]:
feature_extractor.save("extractor_glauk.h5")

In [ ]:
feature_extractor = keras.models.load_model("extractor_glauk.h5")

In [ ]:
feature_extractor.variables

In [ ]:
feature_extractor.summary()

In [ ]:
X = X.reshape(-1, X.shape[3])

In [ ]:
Y = y_train.reshape(-1)

In [ ]:
Y

In [ ]:
dataset = pd.DataFrame(X)
dataset['Label'] = Y
print(dataset['Label'].unique())
print(dataset['Label'].value_counts())

In [ ]:
X_for_RF = dataset.drop(labels = ['Label'], axis=1)
Y_for_RF = dataset['Label']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from joblib import parallel_backend

In [ ]:
model = RandomForestClassifier(n_estimators = 50, max_depth=30, random_state = 42, n_jobs=-1, verbose=1)
trained_model = model.fit(X, Y)

In [ ]:
filename = 'RF_model_glau.sav'
pickle.dump(trained_model, open(filename, 'wb'))

In [ ]:
trained_model = pickle.load(open('RF_model_glau.sav', 'rb'))

In [ ]:
test_img = cv2.imread("test/19_240_XN.jpg ", cv2.IMREAD_COLOR) # 20_000_XN.jpg      11_180_XN.jpg     
# 30_180_XN.jpg    13_180_XN.jpg
test_img = cv2.cvtColor(test_img, cv2.COLOR_RGB2BGR)
plt.imshow(test_img)
plt.show()
test_img = np.expand_dims(test_img, axis=0)

In [ ]:
X_test_feature = feature_extractor.predict(test_img)
X_test_feature = X_test_feature.reshape(-1, X_test_feature.shape[3])

In [ ]:
prediction = trained_model.predict(X_test_feature)

In [ ]:
prediction_image = prediction.reshape(train_masks[0].shape)

In [ ]:
kernel = np.ones((3,3), np.uint8)
imgDilation = cv2.dilate(prediction_image, kernel, iterations = 1)

In [ ]:
plt.imshow(prediction_image, cmap='gray')
plt.show()

In [ ]:
plt.imshow(imgDilation, cmap='gray')
plt.show()

In [ ]:
imgDilation2 = ndimage.median_filter(imgDilation, size=3)

In [ ]:
plt.imshow(imgDilation2, cmap='gray')
plt.show()

In [ ]:
cv2.imwrite('glau.tif', imgDilation2)